In [ ]:
import os
import numpy as np
import pandas as pd
from google.colab import drive
import csv
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Ingredient_QA= pd.read_csv('/content/drive/MyDrive/Ingredient_QA.csv')

In [ ]:
Ingredient_QA.shape

(31264, 2)

In [ ]:
max_len = Ingredient_QA['질문'].apply(lambda x: len(x)).max()
print("max_len:", max_len)
max_len2 = Ingredient_QA['답변'].apply(lambda x: len(x)).max()
print("max_len:", max_len2)

max_len: 63
max_len: 64


In [ ]:
import math
import numpy as np
import pandas as pd
import random
import re
import torch
import urllib.request
from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast
import urllib.request

In [ ]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

Max_len = 70

koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=Max_len):
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        turn = self._data.iloc[idx]
        q = turn['질문']
        q = re.sub(r"([?.!,])", r" ", q)

        a = turn['답변']
        a = re.sub(r"([?.!,])", r" ", a)

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)

        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)

        mask = [1] * len(token_ids)

        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]
            mask += [0]

        return torch.LongTensor(token_ids), torch.LongTensor(mask)


def collate_batch(batch):
    # Find the maximum length in this batch
    max_len = max(len(item[0]) for item in batch)

    # Pad data and mask to have the same length
    padded_data = [item[0].tolist() + [koGPT2_TOKENIZER.pad_token_id] * (max_len - len(item[0])) for item in batch]
    padded_mask = [item[1].tolist() + [0] * (max_len - len(item[1])) for item in batch]

    return torch.LongTensor(padded_data), torch.LongTensor(padded_mask)



train_set = ChatbotDataset(Ingredient_QA, max_len=Max_len)

train_dataloader = DataLoader(train_set, batch_size=4, num_workers=0, shuffle=True, collate_fn=collate_batch)

print("start")
for batch_idx, samples in enumerate(train_dataloader):
    token_ids, mask = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
print("end")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
         48541,  9242,  9379, 13815, 15084,  7801,  8084,   739,    10,     4,
          9548, 32450, 20600, 18101,   401, 10138,  7660,  7663,  7467, 17605,
          7281, 45798,   739,  9407,  8255,  7621,  6853,   739, 10286,  7543,
         27632,  9848,  8539,   739,     1,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2,  9114,  9466,  8397,   739,  9226,  7249,  9810,   739, 35494,
           739,  9040,  8126,   739,  9025,  7763,  8161,  7741,  9379, 36258,
          9242,  9702,  9182, 11909,  9666,  8244,    10,     4,  9548, 32450,
         20600, 18101,   401, 35796,  8052,  8467, 24831,  7991,  8511,   739,
         14330,  6960,  9340, 15475,  7098,  9203, 12438,   739,  9055,  7480,
          6958,   739,     1,     3,     3,     3,     3,     3,     3,     3,
             3,

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import time

Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 학습 데이터 교체할때는 여기 수정
train_set = ChatbotDataset(Ingredient_QA, max_len=Max_len)
train_dataloader = DataLoader(train_set, batch_size=2, num_workers=0, shuffle=True, collate_fn=collate_batch,)

model.to(device)
model.train()

learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 5
Sneg = -1e18
gradient_accumulation_steps = 4

# Loss 기록을 위한 리스트 초기화
losses = []

checkpoint_interval = 1  # 1로 하면 학습 1에폭당 저장, 2로하면 짝수 에폭당 저장.

time1 = time.time()
print("start")
for ep in range(epoch):
    optimizer.zero_grad()
    total_loss = 0

    for step, samples in enumerate(train_dataloader):
        token_ids, mask = samples
        token_ids, mask = token_ids.to(device), mask.to(device)

        outputs = model(token_ids, attention_mask=mask)
        logits = outputs.logits

        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = token_ids[..., 1:].contiguous()

        loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        loss = loss.mean() / gradient_accumulation_steps  # Normalize and ensure scalar
        total_loss += loss.item()
        losses.append(loss.item())  # Loss 기록
        loss.backward()

        # 실제 학습은 gradient_accumulation_steps 의 수에 따라 (지금은 4) 4의 배수일때마다 역전파를 실행한다.
        # 이 말은 4배치 마다 학습을 하겠다는 것이고, 이는 곧 실제 배치 사이즈와 상관없이 4배치 마다 학습을 한다는 것이다.
        # 이는 gpu 메모리가 적을때 굉장히 좋은 옵션이 된다. 실제로는 2배치로 돌리지만 사실상 4배치 8배치 16배치... 로 할 수 있다는 뜻이다.
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            print(f"Step {step+1}, Epoch {ep+1}/{epoch}, Loss: {total_loss}")
            total_loss = 0

    # 매 에폭마다 체크포인트 저장
    if ep % checkpoint_interval == 0:
        # 모델 상태 및 필요한 다른 정보 저장
        checkpoint = {
            'epoch': ep,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'losses': losses,  # 현재까지의 손실 기록 추가
        }
        # 파일로 저장
        torch.save(checkpoint, f"checkpoint_model_epoch_{ep}.pth")

print("end")
time2 = time.time()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Step 11272, Epoch 4/5, Loss: 0.3388432748615742
Step 11276, Epoch 4/5, Loss: 0.42986589670181274
Step 11280, Epoch 4/5, Loss: 0.346787229180336
Step 11284, Epoch 4/5, Loss: 0.3504398241639137
Step 11288, Epoch 4/5, Loss: 0.40401872992515564
Step 11292, Epoch 4/5, Loss: 0.3971584439277649
Step 11296, Epoch 4/5, Loss: 0.3371809870004654
Step 11300, Epoch 4/5, Loss: 0.3126152753829956
Step 11304, Epoch 4/5, Loss: 0.3767300248146057
Step 11308, Epoch 4/5, Loss: 0.30356787517666817
Step 11312, Epoch 4/5, Loss: 0.3024241328239441
Step 11316, Epoch 4/5, Loss: 0.3525763116776943
Step 11320, Epoch 4/5, Loss: 0.32151979953050613
Step 11324, Epoch 4/5, Loss: 0.30277467519044876
Step 11328, Epoch 4/5, Loss: 0.399701863527298
Step 11332, Epoch 4/5, Loss: 0.40999505668878555
Step 11336, Epoch 4/5, Loss: 0.315835177898407
Step 11340, Epoch 4/5, Loss: 0.3092334419488907
Step 11344, Epoch 4/5, Loss: 0.41402407735586166
Step 11348, Epoch 4/5, Loss: 0.34260789304971695

In [ ]:
# 모델 저장
torch.save(model.state_dict(), '/content/drive/MyDrive/Ingredient_base_QA_model.pt')
print("Model saved successfully")

Model saved successfully


In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

class BasedNameModel():
    def __init__(self):
        # 특수 토큰 정의
        self.Q_TKN = "<usr>"
        self.A_TKN = "<sys>"
        self.BOS = '</s>'
        self.EOS = '</s>'
        self.MASK = '<unused0>'
        self.SENT = '<unused1>'
        self.PAD = '<pad>'

        self.tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2',
                                                                 bos_token=self.BOS,
                                                                 eos_token=self.EOS,
                                                                 unk_token="<unk>",
                                                                 pad_token=self.PAD,
                                                                 mask_token=self.MASK)
        self.model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        state_dict = torch.load('/content/drive/MyDrive/Ingredient_base_QA_model.pt', map_location=self.device)
        self.model.load_state_dict(state_dict)
        self.model.to(self.device)

    def predict(self, user_text):
        self.model.eval()
        with torch.no_grad():
          a = ''
          while True:
            input_ids = torch.LongTensor(self.tokenizer.encode(self.Q_TKN + user_text + self.SENT + self.A_TKN + a,
                                                               add_special_tokens=True)).unsqueeze(dim=0)
            input_ids = input_ids.to(self.device)
            outputs = self.model(input_ids)

            logits = outputs.logits

            pred = torch.argmax(logits, dim=-1)
            gen_token = self.tokenizer.convert_ids_to_tokens(pred[0].cpu().numpy().tolist())[-1]
            if gen_token == self.EOS:
              break
            a += gen_token.replace("▁", " ")

        return a.strip()

In [ ]:
# 사용 예시
model = BasedNameModel()
while True:
  user_text = input('사용자: ')
  if user_text == 'quit':
    break
  print('챗봇:', model.predict(user_text))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


사용자: 돼지고기로 만들 수 있는 맛있는 요리를 추천
챗봇: 재료가 가장 많이 포함된 요리: 돼지고기겨자채보쌈  제육보쌈  양장피냉채
사용자: 돼지고기, 고추장
챗봇: 모든 재료가 포함된 요리: 돼지고기겨자채보쌈  제육보쌈
사용자: 돼지고기, 고추장으로 만들 수 있는 요리 추천
챗봇: 재료가 가장 많이 포함된 요리: 제육보쌈  돼지고기김치볶음  사천식자장면
사용자:  돼지고기, 고추장으로 만들 수 있는 요리 추천해줘
챗봇: 재료가 가장 많이 포함된 요리: 제육보쌈  돼지고기김치볶음  사천식자장면
사용자:  돼지고기, 고추장이 들어가는 요리 추천해줄래/
챗봇: 모든 재료가 포함된 요리: 돼지고기김치볶음  제육보쌈
사용자: 새우가 들어가는 요리 추천해줘
챗봇: 모든 재료가 포함된 요리: 사천식 알배추찜  두부새우젓국찌개  새우튀김우동
사용자: 새우, 양파, 마늘, 오징어가 포함된 요리추천
챗봇: 모든 재료가 포함된 요리: 오징어섞어찌개  삼색밀쌈  새우튀김우동
